**Note: This notebook must be changed by the student. As it is now, it does not approximate the solution to the Poisson problem!**

# 2D Poisson Solver on the Unit Square
## ... using Psydac's de Rham interface

In this exercise we write a solver for the 2D Poisson problem on the unit square using Psydac's bilinear form interface.

\begin{align*}
    -\Delta\phi &= f \quad \text{ in }\Omega=]0,1[^2, \\
    \phi &= 0 \quad \text{ on }\partial\Omega,
\end{align*}
for the specific choice
\begin{align*}
    f(x, y) = [ (-x^4 + x^3 + (-y^2 + \pi^2)x^2 + (y^2 - 4y - \pi^2)x + 2y - 2)  \sin(\pi y) + (2\pi x^2(1-x))  \cos(\pi y)]  e^{xy}
\end{align*}

## The Variational Formulation

The corresponding variational formulation reads

\begin{align*}
    \text{Find }\phi\in V\coloneqq H_0^1(\Omega)\text{ such that } \\
    a(\phi, \psi) = l(\psi)\quad\forall\ \psi\in V
\end{align*}

where 
- $a(\phi,\psi) \coloneqq \int_{\Omega} \nabla\phi\cdot\nabla\psi ~ d\Omega$,
- $l(\psi) := \int_{\Omega} f\psi ~ d\Omega$.

## Discrete Model using de Rham objects

In [ ]:
from sympde.calculus    import dot, grad
from sympde.expr        import BilinearForm, LinearForm, integral
from sympde.topology    import elements_of, Square, ScalarFunctionSpace, Derham

domain  = Square('S', bounds1=(0, 1), bounds2=(0, 1))
derham  = Derham(domain, sequence=['h1', 'hcurl', 'l2'])

V0      = derham.V0
V1      = derham.V1

u0, v0  = elements_of(V0, names='u0, v0')
u1, v1  = elements_of(V1, names='u1, v1')

# bilinear form corresponding to V1 mass matrix
m1      = BilinearForm((u1, v1), integral(domain, dot(u1, v1)))

# linear form
from sympy import pi, sin, cos, exp
x,y     = domain.coordinates
f       = ( (-x**4 + x**3 + (-y**2 + pi**2)*x**2 + (y**2 - 4*y - pi**2)*x + 2*y - 2) * sin(pi*y) + (2*pi*x**2*(1-x)) * cos(pi*y) ) * exp(x*y)

l       = LinearForm(v0, integral(domain, f*v0))

In [ ]:
from psydac.api.discretization  import discretize
from psydac.api.settings        import PSYDAC_BACKEND_GPYCCEL

backend     = PSYDAC_BACKEND_GPYCCEL

In [ ]:
ncells  = [16, 16]    # Bspline cells
degree  = [3, 3]      # Bspline degree

In [ ]:
domain_h    = discretize(domain, ncells=ncells, periodic=[False, False])
derham_h    = discretize(derham, domain_h, degree=degree)

V0_h        = derham_h.V0
V1_h        = derham_h.V1

# Exterior Derivative operators (grad and curl)
G, C        = derham_h.derivatives_as_matrices

# Mass Matrix
m1_h        = discretize(m1,    domain_h, (V1_h, V1_h), backend=backend)

M1          = m1_h.assemble()

# System Matrix A
from psydac.linalg.basic import IdentityOperator
A           = G.T @ M1 @ G

# rhs vector f
l_h         = discretize(l, domain_h, V0_h, backend=backend)
f           = l_h.assemble()

## Boundary Conditions

We choose to apply a projection method. For that matter, we construct the projection matrix $\mathbb{P}_0$ and its counterpart $\mathbb{P}_{\Gamma} = \mathbb{I} - \mathbb{P}_0$.

In [ ]:
from utils import H1BoundaryProjector2D
from psydac.linalg.basic import IdentityOperator

P_0     = H1BoundaryProjector2D(V0, V0_h.coeff_space)

I_0     = IdentityOperator(V0_h.coeff_space)
P_Gamma = I_0 - P_0

In [ ]:
A_bc    = P_0 @ A @ P_0 + P_Gamma
f_bc    = P_0 @ f

## Solving the PDE

In [ ]:
import  time

from    psydac.linalg.solvers import inverse

tol     = 1e-12
maxiter = 1000

A_bc_inv = inverse(A_bc, 'cg', tol=tol, maxiter=maxiter)

t0      = time.time()
phi_h   = A_bc_inv @ f_bc
t1      = time.time()

## Computing the error norm

When the analytical solution is available, you might be interested in computing the $L^2$ norm or $H^1_0$ semi-norm.
SymPDE allows you to do so, by creating the **Norm** object.
In this example, the analytical solution is given by

$$
phi_{ex}(x, y) = x(x-1)\sin(\pi y)e^{xy}
$$

### Computing the $L^2$ norm

In [ ]:
from psydac.fem.basic   import FemField

from sympde.expr        import Norm, SemiNorm

phi_ex = x*(x-1)*sin(pi*y)*exp(x*y)
phi_h_FemField = FemField(V0_h, phi_h)

error = phi_ex - u0

# create the formal Norm object
l2norm = Norm(error, domain, kind='l2')

# discretize the norm
l2norm_h = discretize(l2norm, domain_h, V0_h, backend=backend)

# assemble the norm
l2_error = l2norm_h.assemble(u0=phi_h_FemField)

### Computing the $H^1$ semi-norm

In [ ]:
# create the formal Norm object
h1norm = SemiNorm(error, domain, kind='h1')

# discretize the norm
h1norm_h = discretize(h1norm, domain_h, V0_h)

# assemble the norm
h1semi_error = h1norm_h.assemble(u0=phi_h_FemField)

### Computing the $H^1$ norm

In [ ]:
# create the formal Norm object
h1norm = Norm(error, domain, kind='h1')

# discretize the norm
h1norm_h = discretize(h1norm, domain_h, V0_h)

# assemble the norm
h1_error = h1norm_h.assemble(u0=phi_h_FemField)

# print the result
print( '> Grid          :: [{ne1},{ne2}]'.format( ne1=ncells[0], ne2=ncells[1]) )
print( '> Degree        :: [{p1},{p2}]'  .format( p1=degree[0], p2=degree[1] ) )
print( '> CG info       :: ',A_bc_inv.get_info() )
print( '> L2 error      :: {:.2e}'.format( l2_error ) )
print( '> H1-Semi error :: {:.2e}'.format( h1semi_error ) )
print( '> H1 error      :: {:.2e}'.format( h1_error ) )
print( '' )
print( '> Solution time :: {:.3g}'.format( t1-t0 ) )

## Visualization

We plot the true solution $\phi_{ex}$, the approximate solution $\phi_h$ and the error function $|\phi_{ex} - \phi_h|$.

In [ ]:
from sympy import lambdify

from utils import plot

phi_ex_fun = lambdify(domain.coordinates, phi_ex)
error = lambda x, y: abs(phi_ex_fun(x, y) - phi_h_FemField(x, y))

plot(gridsize_x   = 100, 
     gridsize_y    = 100, 
     title         = r'Approximation of Solution $\phi$', 
     funs          = [phi_ex_fun, phi_h_FemField, error], 
     titles        = [r'$\phi_{ex}(x,y)$', r'$\phi_h(x,y)$', r'$|(\phi_{ex}-\phi_h)(x,y)|$'],
     surface_plot  = True
)